In [39]:
!pip install torch

In [40]:
import urllib.request 
url=("https://raw.githubusercontent.com/rasbt/"
"LLMs-from-scratch/main/ch02/01_main-chapter-code/"
"the-verdict.txt")
file_path="the-verdict.txt"
urllib.request.urlretrieve(url,file_path)

('the-verdict.txt', <http.client.HTTPMessage at 0x127b0d250>)

In [41]:
with open ("the-verdict.txt","r",encoding="utf-8") as f:
    raw_text=f.read()
print("Total number of character:",len(raw_text))
print(raw_text[:99])    

Total number of character: 20479
I HAD always thought Jack Gisburn rather a cheap genius--though a good fellow enough--so it was no 


In [42]:
import re 
text="HELLO, I AM BHAVYA GOYAL !"
result=re.split(r'(\s)',text)
result

['HELLO,', ' ', 'I', ' ', 'AM', ' ', 'BHAVYA', ' ', 'GOYAL', ' ', '!']

In [43]:
result = re.split(r'([,.]|\s)', text)
print(result)

['HELLO', ',', '', ' ', 'I', ' ', 'AM', ' ', 'BHAVYA', ' ', 'GOYAL', ' ', '!']


In [44]:
result=[item for item in result if item.strip()]
result

['HELLO', ',', 'I', 'AM', 'BHAVYA', 'GOYAL', '!']

In [45]:
text="hello my name is bhavya goyal I am intested in deep generative models "
result = re.split(r'([,.:;?_!"()\']|--|\s)', text)
result = [item.strip() for item in result if item.strip()]
print(result)

['hello', 'my', 'name', 'is', 'bhavya', 'goyal', 'I', 'am', 'intested', 'in', 'deep', 'generative', 'models']


In [46]:
preprocessed=re.split(r'([,.:;?_!"()\']|--|\s)', raw_text)
preprocessed=[item for item in preprocessed if item.strip()]
print(len(preprocessed))
len(set(preprocessed))


4690


1130

In [47]:
import torch

if torch.backends.mps.is_available():
    print("MPS is available!")
else:
    print("MPS is not available.")


MPS is available!


In [48]:
device = torch.device("mps" if torch.backends.mps.is_available() else "cpu")
print(device)

mps


In [49]:
all_words=sorted(set(preprocessed))
vocab_size=len(all_words)
print(vocab_size)

1130


In [50]:
vocab={token:integer for integer,token in enumerate(all_words)}
for i,item in enumerate(vocab.items()):
    print(item)
    if i>50:
        break


('!', 0)
('"', 1)
("'", 2)
('(', 3)
(')', 4)
(',', 5)
('--', 6)
('.', 7)
(':', 8)
(';', 9)
('?', 10)
('A', 11)
('Ah', 12)
('Among', 13)
('And', 14)
('Are', 15)
('Arrt', 16)
('As', 17)
('At', 18)
('Be', 19)
('Begin', 20)
('Burlington', 21)
('But', 22)
('By', 23)
('Carlo', 24)
('Chicago', 25)
('Claude', 26)
('Come', 27)
('Croft', 28)
('Destroyed', 29)
('Devonshire', 30)
('Don', 31)
('Dubarry', 32)
('Emperors', 33)
('Florence', 34)
('For', 35)
('Gallery', 36)
('Gideon', 37)
('Gisburn', 38)
('Gisburns', 39)
('Grafton', 40)
('Greek', 41)
('Grindle', 42)
('Grindles', 43)
('HAD', 44)
('Had', 45)
('Hang', 46)
('Has', 47)
('He', 48)
('Her', 49)
('Hermia', 50)
('His', 51)


In [51]:
import re

class SimpleTokenizerV1:
    def __init__(self, vocab):
        self.str_to_int = vocab
        self._int_to_str = {i: s for s, i in vocab.items()}
    
    def encode(self, text):
        # Using 'text' instead of 'raw_text'
        preprocessed = re.split(r'([,.:;?_!"()\']|--|\s)', text)
        preprocessed = [item.strip() for item in preprocessed if item.strip()]
        ids = [self.str_to_int[s] for s in preprocessed]
        return ids
    
    def decode(self, ids):
        # Using '_int_to_str' instead of 'int_to_str'
        text = " ".join([self._int_to_str[i] for i in ids])
        print(text)
        text = re.sub(r'\s+([,.?!"()\'\\])', r'\1', text)
        return text


In [52]:
tokenizer=SimpleTokenizerV1(vocab)

In [53]:
tokenizer.encode("my,is ")

[697, 5, 584]

In [54]:
tokenizer.decode([697, 5,584])

my , is


'my, is'

In [55]:
tokenizer.encode("hi my name is bhavya ")

KeyError: 'hi'

In [56]:
all_tokens=sorted(list(set(preprocessed)))
all_tokens.extend(['<|unk|>','<|endoftext|>'])
vocab={token:integer for integer,token in enumerate(all_tokens)}
print(len(vocab.items()))

1132


In [57]:
for i , item in enumerate(list(vocab.items())[-5:]):
    print(item)

('younger', 1127)
('your', 1128)
('yourself', 1129)
('<|unk|>', 1130)
('<|endoftext|>', 1131)


In [62]:
import re

class SimpleTokenizerV2:
    def __init__(self, vocab):
        self.str_to_int = vocab
        self.int_to_str = {i: s for s, i in vocab.items()}
    
    def encode(self, text):
        preprocessed = re.split(r'([,.:;?_!"()\']|--|\s)', text)
        preprocessed = [item.strip() for item in preprocessed if item.strip()]
        preprocessed = [item if item in self.str_to_int else "<|unk|>" for item in preprocessed]
        ids = [self.str_to_int[s] for s in preprocessed]
        return ids
    
    def decode(self, ids):
        text = " ".join([self.int_to_str[i] for i in ids])
        text = re.sub(r'\s+([,.:;?!()\'"])', r'\1', text)
        return text



In [66]:
test="hi my name is bhavya goyal my biggest regret is pursuing btech"
tokenizer=SimpleTokenizerV2(vocab)
print(tokenizer.encode(test))
print(tokenizer.decode(tokenizer.encode(test)))

[1130, 697, 1130, 584, 1130, 1130, 697, 1130, 1130, 584, 1130, 1130]
<|unk|> my <|unk|> is <|unk|> <|unk|> my <|unk|> <|unk|> is <|unk|> <|unk|>


In [79]:
text1="My name is BHAVYA GOYAL I am currently a sophomore."
text2="My dog's name is BRUNO."
concatenated_text=" <|endoftext|> ".join([text1,text2])
print(concatenated_text)

My name is BHAVYA GOYAL I am currently a sophomore. <|endoftext|> My dog's name is BRUNO.


In [80]:
print(tokenizer.encode(concatenated_text))

[68, 1130, 584, 1130, 1130, 53, 150, 1130, 115, 1130, 7, 1131, 68, 1130, 2, 850, 1130, 584, 1130, 7]


In [81]:
print(tokenizer.decode(tokenizer.encode(concatenated_text)))

My <|unk|> is <|unk|> <|unk|> I am <|unk|> a <|unk|>. <|endoftext|> My <|unk|>' s <|unk|> is <|unk|>.


In [82]:
!pip install tiktoken

  Using cached tiktoken-0.8.0-cp312-cp312-macosx_11_0_arm64.whl.metadata (6.6 kB)
Using cached tiktoken-0.8.0-cp312-cp312-macosx_11_0_arm64.whl (982 kB)


In [83]:
from importlib.metadata import version
import tiktoken
print('TIKTOKEN VERISON:',version('tiktoken'))

TIKTOKEN VERISON: 0.8.0


In [87]:
bpe_tokenizer=tiktoken.get_encoding('gpt2')

In [92]:
text1="My name is BHAVYA GOYAL I am currently a sophomore."
text2="My dog's name is BRUNO."
concatenated_text=" <|endoftext|> ".join([text1,text2])
print(bpe_tokenizer.encode(concatenated_text,allowed_special={"<|endoftext|>"}))
print(tokenizer.decode(bpe_tokenizer.encode(concatenated_text,allowed_special={"<|endoftext|>"})))

[3666, 1438, 318, 347, 7801, 53, 44947, 10351, 56, 1847, 314, 716, 3058, 257, 24505, 13, 220, 50256, 2011, 3290, 338, 1438, 318, 11177, 4944, 46, 13]
My name is BHAVYA GOYAL I am currently a sophomore. <|endoftext|> My dog's name is BRUNO.


In [95]:
with open('the-verdict.txt','r') as f:
    content=f.read()
encoded_text=tokenizer.encode(content)  
print(len(encoded_text))


5145


In [97]:
print(tokenizer.decode(encoded_text[:10]))

I HAD always thought Jack Gisburn rather


In [108]:
context_size=5
x=encoded_text[:5]
y=encoded_text[1:6]
print(f'input to the model:{x}')
print(f'labels for the model: {y}')
print(tokenizer.decode(encoded_text[:5]))
print(tokenizer.decode(encoded_text[1:6]))




input to the model:[40, 367, 2885, 1464, 1807]
labels for the model: [367, 2885, 1464, 1807, 3619]
I HAD always thought
 HAD always thought Jack


In [123]:
for i in range(1,context_size+1):
    context=encoded_text[:i]
    desired=encoded_text[i]
    print(f'{context}---->{desired}')

[40]---->367
[40, 367]---->2885
[40, 367, 2885]---->1464
[40, 367, 2885, 1464]---->1807
[40, 367, 2885, 1464, 1807]---->3619


In [126]:
for i in range(1,context_size+1):
    context=encoded_text[:i]
    
    
    desired=encoded_text[i]
    
    print(f'{bpe_tokenizer.decode(context)}---->{bpe_tokenizer.decode([desired])}')

I----> H
I H---->AD
I HAD----> always
I HAD always----> thought
I HAD always thought----> Jack


In [133]:
import torch
from torch.utils.data import DataLoader,Dataset
class GPTDatasetV1(Dataset):
    def __init__(self,text,tokenizer,max_length,stride):
        # super().__init_()
        self.input_ids=[]
        self.target_ids=[]

        token_ids=bpe_tokenizer.encode(text)


        for i in range(0,len(token_ids)-max_length,stride):
            input_chunk=token_ids[i:i+max_length]
            target_chunk=token_ids[i+1:i+max_length+1]
            self.input_ids.append(torch.tensor(input_chunk))
            self.target_ids.append(torch.tensor(target_chunk))

    def __len__(self):
        return len(self.input_ids)

    def __getitem__(self, idx):
        return self.input_ids[idx],self.target_ids[idx]
            





In [136]:
def create_dataloader_v1(txt,batch_size=4,max_length=267,
                         stride=128,shuffle=True,drop_last=True,
                         num_workers=0):
    bpe_tokenizer=tiktoken.get_encoding('gpt2')
    dataset=GPTDatasetV1(txt,bpe_tokenizer,max_length,stride)
    dataloader=DataLoader(dataset,
        batch_size=batch_size,
        shuffle=shuffle,
        drop_last=drop_last,
        num_workers=num_workers)
    
    return dataloader


In [150]:
torch.manual_seed(2000)
with open("the-verdict.txt",'r',encoding='utf-8') as f:
    raw_text=f.read()
dataloader=create_dataloader_v1(raw_text,batch_size=4,max_length=4
                                ,stride=3
                                ,shuffle=False)

data_iter=iter(dataloader)

first_batch=next(data_iter)
print(first_batch)

[tensor([[   40,   367,  2885,  1464],
        [ 1464,  1807,  3619,   402],
        [  402,   271, 10899,  2138],
        [ 2138,   257,  7026, 15632]]), tensor([[  367,  2885,  1464,  1807],
        [ 1807,  3619,   402,   271],
        [  271, 10899,  2138,   257],
        [  257,  7026, 15632,   438]])]


In [155]:
torch.manual_seed(2000)
with open("the-verdict.txt",'r',encoding='utf-8') as f:
    raw_text=f.read()
dataloader=create_dataloader_v1(raw_text,batch_size=8,max_length=4
                                ,stride=4
                                ,shuffle=False)

data_iter=iter(dataloader)

input_batch,target_batch=next(data_iter)
print(f'input batch----->\n{input_batch}')
print(f'targer batch----->\n{target_batch}')

input batch----->
tensor([[   40,   367,  2885,  1464],
        [ 1807,  3619,   402,   271],
        [10899,  2138,   257,  7026],
        [15632,   438,  2016,   257],
        [  922,  5891,  1576,   438],
        [  568,   340,   373,   645],
        [ 1049,  5975,   284,   502],
        [  284,  3285,   326,    11]])
targer batch----->
tensor([[  367,  2885,  1464,  1807],
        [ 3619,   402,   271, 10899],
        [ 2138,   257,  7026, 15632],
        [  438,  2016,   257,   922],
        [ 5891,  1576,   438,   568],
        [  340,   373,   645,  1049],
        [ 5975,   284,   502,   284],
        [ 3285,   326,    11,   287]])


In [156]:
input_ids=torch.tensor([0,1,2,3])


In [157]:
vocab_size=6
output_dim=3


In [158]:
torch.manual_seed(100)
embedding_layer=torch.nn.Embedding(vocab_size,output_dim)
print(embedding_layer.weight)

Parameter containing:
tensor([[ 0.1268,  1.3564, -0.0247],
        [-0.8466,  0.0293, -0.5721],
        [-1.2546,  0.0486,  0.2753],
        [-2.1550, -0.7116,  0.0575],
        [ 0.6263, -1.7736, -0.2205],
        [ 2.7467, -1.0480,  1.1239]], requires_grad=True)


In [159]:
embedding_layer(input_ids)

tensor([[ 0.1268,  1.3564, -0.0247],
        [-0.8466,  0.0293, -0.5721],
        [-1.2546,  0.0486,  0.2753],
        [-2.1550, -0.7116,  0.0575]], grad_fn=<EmbeddingBackward0>)

In [162]:
vocab_size=50257
output_dim=256
token_embedding_layer=torch.nn.Embedding(vocab_size,output_dim)
print(token_embedding_layer)

Embedding(50257, 256)


In [167]:
max_lenth=4
dataloader=create_dataloader_v1(
    raw_text,batch_size=8,stride=max_lenth,max_length=max_lenth,
    shuffle=False
)
data_iter=iter(dataloader)
input_batch,target_batch=next(data_iter)
print(f'input batch:{input_batch.shape}')

input batch:torch.Size([8, 4])


In [170]:
token_embeddings=token_embedding_layer(input_batch)
token_embeddings.shape

torch.Size([8, 4, 256])

In [184]:
pos_embedding=torch.nn.Embedding(max_lenth,output_dim)
pos_embedding=pos_embedding(torch.arange(0,max_lenth))
pos_embedding.shape


torch.Size([4, 256])

In [186]:
input_embeddings=token_embeddings+pos_embedding
print(input_embeddings.shape)

torch.Size([8, 4, 256])
